In [95]:
def transform_table1(table):
    cleaned_data = []
    for row in table:
        transformed_row = [transform_expression(item) for item in row]
        transforemd_row = [transform_acceleration(item) for item in transformed_row]
        cleaned_data.append(transformed_row)
    return cleaned_data

def transform_table2(table):
    cleaned_data = []
    for row in table:
        cleaned_row = [item.replace('\n', '') if isinstance(item, str) else item for item in row]
        transformed_row = [transform_expression(item) for item in row]
        transforemd_row = [transform_acceleration(item) for item in transformed_row]
        cleaned_data.append(transformed_row)
    return cleaned_data


def transform_header(cleaned_data):
    header_index = 0
    for i,row in enumerate(cleaned_data):
        if any("Column" in str(item) for item in row):
            header_index = i
            break
    return cleaned_data[header_index]

def transform_expression(expression):
    if isinstance(expression, str):
        # 'V2'를 'V^2'로 변경
        return re.sub(r'\bV2\b', 'V^2', expression)
    return expression

def transform_acceleration(acceleration):
    if isinstance(acceleration, str):
        # 'm/s2'를 'm/s^2'로 변경
        return re.sub(r'\bm/s2\b', 'm/s^2', acceleration)
    return acceleration

In [115]:
import re
import pdfplumber
import pandas as pd

file_path = "Merge_rel.pdf"
result_dict = {}  # dict 초기화
with pdfplumber.open(file_path) as pdf:
    temp_table = None  # temp_table 초기화
    for num in range(len(pdf.pages)):
        page = pdf.pages[num]
        text = page.extract_text()
        
        # rel_match 추출
        rel_matches = re.findall(r"Addendum\s\d+\s*[:–-]\s*UN\sRegulation\sNo\.\s\d+", text)
        if rel_matches:
            rel_match = rel_matches[0]
            if rel_match not in result_dict:
                result_dict[rel_match] = {}
        
        tables = page.extract_tables()
        df = None  # df 초기화
        table = None  # table 초기화
        
        if num == 50:
            for i in range(len(tables)):
                table = tables[i]
                if i == 0:
                    if len(table) == 2:
                        cleaned_data = transform_table1(table)
                    else:
                        cleaned_data = transform_table2(table)
                        header = transform_df2(cleaned_data)
                    df = pd.DataFrame(cleaned_data[1:], columns=cleaned_data[0])
                else:
                    temp_table = table
        elif num == 51:
            for i in range(len(tables)):
                table = tables[i]
                if temp_table is not None:
                    table = temp_table + table  # 테이블 병합
                if len(table) == 2:
                    cleaned_data = transform_table1(table)
                else:
                    cleaned_data = transform_table2(table)
                    header = transform_header(cleaned_data)
                df = pd.DataFrame(cleaned_data[1:], columns=cleaned_data[0])
        else:
            if tables:
                table = tables[0]
                if len(table) == 2:
                    cleaned_data = transform_table1(table)
                else:
                    cleaned_data = transform_table2(table)
                    header = transform_df2(cleaned_data)
                df = pd.DataFrame(cleaned_data[1:], columns=cleaned_data[0])
        
        # markdown 변환 및 저장
        if df is not None:
            markdown_df = df.to_markdown()
            table_name_matches = re.findall("Table [0-9]{1,}", text)
            if table_name_matches:
                table_name = table_name_matches[0]
                result_dict[rel_match][table_name] = {
                    "location": num + 1,
                    "content": markdown_df
                }
            else:
                result_dict[rel_match][f"unknown_{num}"] = {
                    "location": num + 1,
                    "content": markdown_df
                }
with open('table.json', 'w') as f:
    json.dump(result_dict, f, indent=4, ensure_ascii=False)
